# Logistic classifier applied to text data



The goal of this notebook is to explore logistic regression and feature engineering with existing Turi Create functions.

We will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

The notes follow the Classification course from UW on Coursera.
    

In [4]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

# Logistic classifier notes

A logistic classifier uses regression to model a discrete target variable as a function of several feature variables, i.e. predictors.

The logistic classifier uses a discrete target variable $Y$ instead of a scalar. _For each observation, the probability that $Y=1$ (instead of $0$) is modeled as the logistic function of a linear combination of the feature values_.

## Assumptions

We have the following two assumptions for logistic regression:

0. Let the independent variables (features) $x^i$ not be multilinear, i.e., they are linearly independent.


1. Let $Y \in \left\{0,1\right\}$ and $p:= P(Y=1)$ satify. Assume the _log-odds_ satisfy:
\begin{align}\label{assumption}
\log_b \left( \frac{p}{1-p}\right) = \beta_i x^i
\end{align}
where $\beta_i$ constants, and $x^0=1$.

 Then above statement that the probability is modeled as a logistic function of a linear combination of the features holds:

\begin{align}
\frac{p}{1-p} = b^{\beta_i x^i} \ , \\
\frac{1-p}{p} = b^{-\beta_i x^i} \ , \\
p = \frac{1}{1+b^{-\beta_i x^i}} \ .
\end{align}



Allowing for some feature engineering we have the following more general case of (\ref{assumption}) :

\begin{align}\label{assumption2}
\log_b \left( \frac{p}{1-p}\right) = \beta_i h^i(\underline x) 
\end{align}
where $\underline x = (x^1,x^2,\dots , x^N)$ are the underlying independent variables.

We call the RHS of the above the _score_ : $\operatorname{score}(\underline x):= \beta_i h^i(\underline x)$.

####  Example

Take $b=10, \beta_0=-3,\beta_1=1,\beta_2=2$ in (\ref{assumption}).

- $\beta_0 = -3$ means at that at event $(x^0, x^1) = (0,0)$, the odds are $10^{-3}$ or 1-to-1000. The probability is $p=1/(1 + 10^3) = 1/1001$.


- $\beta_2 = 2$ means $x_2 \rightarrow x_2 +1$ then log-odds increase by 2, and the odds increase by $10^2$.


## Cost function

The cost function for the logistic regression problem is the likelihood function:

\begin{align}
p_\theta(x) = \frac{1}{1 + e^{-\beta_i x^i}} = \Pr(Y=1 \mid x; \beta)
\end{align}

Therefore,
\begin{align} 
\Pr(Y=0 \mid x; \beta) = 1 - p_\beta(x) \ .
\end{align}

and since $Y \in \{0,1\}$, we see that  $\Pr(Y\mid x;\beta)$ is given by 
\begin{align}
\Pr(Y \mid x; \beta) = h_\beta(x)^Y(1 - h_\beta(x))^{(1-Y)} \ .
\end{align}

__For $\underline Y$ we write $\Pr(Y_i\mid x^{(i)};\beta)$__ .

We now calculate the likelihood function assuming that all the observations in the sample are independently Bernoulli distributed,

\begin{align}
L(\beta \mid x) &= \Pr(\underline Y \mid X; \beta) \\
 & = \prod_i \Pr(Y_i \mid x^{(i)}; \beta) \\
 & = \prod_i p_\beta(x^{(i)})^{Y_i}(1 - p_\beta(x^{(i)}))^{(1-Y_i)} \ .
\end{align}

Typically, the log likelihood is maximized,

\begin{align}
N^{-1} \log L(\theta \mid x) = N^{-1} \sum_{i=1}^N \log \Pr(y_i \mid x^{(i)}; \theta)
\end{align}
which is maximized using optimization techniques such as gradient descent.

### Import Turi Create

In [46]:
from __future__ import division
import turicreate
import math
import string
import numpy as np

# Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [2]:
products = turicreate.SFrame('amazon_baby.sframe/')

Now, let us see a preview of what the dataset looks like.

In [4]:
products.head()

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [5]:
products[269]

{'name': 'The First Years Massaging Action Teether',
 'rating': 5.0,
 'review': 'A favorite in our house!'}

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [6]:
import string 
def remove_punctuation(text):
    try: # python 2.x
        text = text.translate(None, string.punctuation) 
    except: # python 3.x
        translator = text.maketrans('', '', string.punctuation)
        text = text.translate(translator)
    return text

review_without_punctuation = products['review'].apply(remove_punctuation)
products['word_count'] = turicreate.text_analytics.count_words(review_without_punctuation)

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [7]:
products[269]['word_count']

{'a': 1, 'favorite': 1, 'house': 1, 'in': 1, 'our': 1}

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [8]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [9]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3, 'love': 1,'it': 3, 'highly': 1, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'and': 3, 'ingenious':1, 'love': 2, 'what': 1, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2, 'all': 2,'help': 1, 'cried': 1, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2, 'this': 2,'her': 1, 'help': 2, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1, 'noble': 1,'is': 1, 'it': 1, 'as': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2, 'all': 1,'right': 1, 'had': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1, 'fantastic':1, 'help': 1, 'give': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'all': 1, 'standarad':1, 'another': 1, 'when': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 2, 'nannys': 1,'just': 1, 'food': 1, ...",1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [10]:
train_data, test_data = products.random_split(.8, seed=1)
print(len(train_data))
print(len(test_data))

133416
33336


# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

In [11]:
sentiment_model = turicreate.logistic_classifier.create(train_data,
                                                        target = 'sentiment',
                                                        features=['word_count'],
                                                        validation_set=None)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 2         | 11       | 2.778177  | 4.053210     | 0.942638          |

| 3         | 12       | 2.778177  | 4.347708     | 0.967822          |

| 4         | 13       | 2.778177  | 4.632210     | 0.976495          |

| 5         | 14       | 2.778177  | 4.935276     | 0.976495          |

+-----------+----------+-----------+--------------+-------------------+

In [12]:
sentiment_model

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 121713
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 121712

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : lbfgs
Solver iterations              : 5
Solver status                  : TERMINATED: Terminated due to numerical difficulties.
Training time (sec)            : 3.2825

Settings
--------
Log-likelihood                 : inf

Highest Positive Coefficients
-----------------------------
word_count[offsi]              : 21.7657
word_count[featurebelt]        : 21.6818
word_count[articletest]        : 21.6818
word_count[conclusions]        : 21.6818
word_count[regulatetest]       : 21.6818

Lowest Negative Coefficients
----------------------------
word_count[lactina

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

In [13]:
weights = sentiment_model.coefficients
weights.column_names()

['name', 'index', 'class', 'value', 'stderr']

There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [21]:
num_positive_weights = len(weights[weights['value']>= 0])
num_negative_weights = len(weights[weights['value']<0])

print("Number of positive weights: %s " % num_positive_weights)
print("Number of negative weights: %s " % num_negative_weights)

Number of positive weights: 91073 
Number of negative weights: 30640 


## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [22]:
sample_test_data = test_data[10:13]
print(sample_test_data['rating'])
sample_test_data

[5.0, 2.0, 1.0]


name,review,rating,word_count,sentiment
Our Baby Girl Memory Book,Absolutely love it andall of the Scripture in ...,5.0,"{'and': 2, 'all': 1,'love': 1, 'purchased': ...",1
Wall Decor RemovableDecal Sticker - Colorful ...,Would not purchase againor recommend. The decals ...,2.0,"{'and': 1, 'would': 2,'almost': 1, 'decals' ...",-1
New Style Trailing CherryBlossom Tree Decal ...,Was so excited to getthis product for my baby ...,1.0,"{'all': 1, 'money': 1,'into': 1, 'back': 1, ...",-1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [23]:
sample_test_data[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [24]:
sample_test_data[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using Turi Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [25]:
scores = sentiment_model.predict(sample_test_data, output_type='margin')
print(scores)

[4.7889073092101455, -3.00078222244489, -8.188501360763553]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [41]:
def class_prediction(scores):
    
    score_sf = turicreate.SFrame(scores)
    preds = score_sf['X1'].apply(lambda X1 : 1 if X1 > 0 else -1)
    return preds

In [42]:
class_prediction(scores)

dtype: int
Rows: 3
[1, -1, -1]

Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from Turi Create.

In [43]:
print("Class predictions according to Turi Create:")
print(sentiment_model.predict(sample_test_data))

Class predictions according to Turi Create:
[1, -1, -1]


**Checkpoint**: Make sure your class predictions match with the one obtained from Turi Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [53]:
def prob_prediction(scores):
    
    probs = 1/(1 + np.exp(-1*scores))
    
    return probs

In [55]:
prob_prediction(scores)

array([9.91747131e-01, 4.73905475e-02, 2.77752771e-04])

**Checkpoint**: Make sure your probability predictions match the ones obtained from Turi Create.

In [56]:
print("Class predictions according to Turi Create:")
print(sentiment_model.predict(sample_test_data, output_type='probability'))

Class predictions according to Turi Create:
[0.9917471313286568, 0.047390547487917316, 0.0002777527712170432]


### Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use Turi Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, we find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, we use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. One can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.

In [57]:
test_data_prob = sentiment_model.predict(test_data,output_type='probability')

In [71]:
result_test = test_data.copy()
result_test['sentiment_model_prob'] = test_data_prob
result_test.topk(column_name='sentiment_model_prob',k=20)['name']

dtype: str
Rows: 20
['Britax Decathlon Convertible Car Seat, Tiffany', "Lilly Gold Sit 'n' Stroll 5 in 1 Car Seat and Stroller Combination, Tuxedo Black (sunshade is not included in the offering)", 'Fisher-Price Rainforest Melodies and Lights Deluxe Gym', 'Peg Perego Aria Light Weight One Hand Fold Stroller in Moka', "P'Kolino Silly Soft Seating in Tias, Green", 'Evenflo X Sport Plus Convenience Stroller - Christina', 'Britax Frontier Booster Car Seat', 'Baby Einstein Around The World Discovery Center', 'Infantino Wrap and Tie Baby Carrier, Black Blueberries', 'bumGenius One-Size Snap Closure Cloth Diaper 4.0 - Bubble', "Fisher-Price Cradle 'N Swing,  My Little Snugabunny", 'Roan Rocco Classic Pram Stroller 2-in-1 with Bassinet and Seat Unit - Coffee', 'Quinny 2012 Buzz Stroller, Rebel Red', 'Britax 2012 B-Agile Stroller, Red', 'Maxi-Cosi Pria 70 with Tiny Fit Convertible Car Seat', "Graco Pack 'n Play Element Playard - Flint", 'Diono RadianRXT Convertible Car Seat, Plum', 'Graco FastA

Now, let us repeat this exercise to find the "most negative reviews." We use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. 

In [73]:
result_test.topk(column_name='sentiment_model_prob',k=20,reverse=True)['name']

dtype: str
Rows: 20
['Luna Lullaby Bosom Baby Nursing Pillow, Chocolate Dot', 'The First Years True Choice P400 Premium Digital Monitor, 2 Parent Unit', 'Jolly Jumper Arctic Sneak A Peek Infant Car Seat Cover Black', 'Motorola MBP36 Remote Wireless Video Baby Monitor with 3.5-Inch Color LCD Screen, Infrared Night Vision and Remote Camera Pan, Tilt, and Zoom', 'VTech Communications Safe &amp; Sounds Full Color Video and Audio Monitor', 'Fisher-Price Ocean Wonders Aquarium Bouncer', "Levana Safe N'See Digital Video Baby Monitor with Talk-to-Baby Intercom and Lullaby Control (LV-TW501)", 'Safety 1st High-Def Digital Monitor', 'Snuza Portable Baby Movement Monitor', 'Adiri BPA Free Natural Nurser Ultimate Bottle Stage 1 White, Slow Flow (0-3 months)', 'Samsung SEW-3037W Wireless Pan Tilt Video Baby Monitor Infrared Night Vision and Zoom, 3.5 inch', 'Motorola Digital Video Baby Monitor with Room Temperature Thermometer', 'Cloth Diaper Sprayer--styles may vary', 'Munchkin Nursery Projector a

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions.
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

In [113]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    ## YOUR CODE HERE
    
    correct_classify = (len(np.nonzero(np.array(model.predict(data) == true_labels))[0]))
    accuracy = correct_classify / len(test_data)
    
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    

    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE

    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [114]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.9221862251019919

### Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [115]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [116]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [117]:
train_data

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3, 'love': 1,'it': 3, 'highly': 1, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'and': 3, 'ingenious':1, 'love': 2, 'what': 1, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2, 'all': 2,'help': 1, 'cried': 1, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2, 'this': 2,'her': 1, 'help': 2, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1, 'noble': 1,'is': 1, 'it': 1, 'as': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2, 'all': 1,'right': 1, 'had': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1, 'fantastic':1, 'help': 1, 'give': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This book is perfect!I'm a first time new ...,5.0,"{'and': 1, 'is': 1, 'in':1, 'it': 1, 'im': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I originally just gavethe nanny a pad of paper ...,4.0,"{'all': 1, 'just': 1,'not': 1, 'isnt': 1, ...",1


In [118]:
train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

Let's see what the first example of the dataset looks like:

In [119]:
train_data[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

The **word_count** column had been working with before looks like the following:

In [120]:
print(train_data[0]['word_count'])

{'and': 3, 'love': 1, 'it': 3, 'highly': 1, 'osocozy': 1, 'bags': 1, 'holder': 1, 'leak': 1, 'moist': 1, 'does': 1, 'recommend': 1, 'was': 1, 'wipes': 1, 'early': 1, 'not': 2, 'now': 1, 'disappointed': 1, 'wipe': 1, 'keps': 1, 'wise': 1, 'i': 1, 'planet': 1, 'my': 2, 'came': 1}


Since we are only working with a subset of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [121]:
print(train_data[0]['word_count_subset'])

{'love': 1, 'disappointed': 1}


## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [122]:
simple_model = turicreate.logistic_classifier.create(train_data,
                                                     target = 'sentiment',
                                                     features=['word_count_subset'],
                                                     validation_set=None)
simple_model

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 6        | 0.820049     | 0.866815          |

| 6         | 7        | 0.949303     | 0.866815          |

+-----------+----------+--------------+-------------------+

SUCCESS: Optimal solution found.

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 21
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 20

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 6
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.9641

Settings
--------
Log-likelihood                 : 44323.7254

Highest Positive Coefficients
-----------------------------
word_count_subset[loves]       : 1.6773
word_count_subset[perfect]     : 1.5145
word_count_subset[love]        : 1.3654
(intercept)                    : 1.2995
word_count_subset[easy]        : 1.1937

Lowest Negative Coefficients
----------------------------
word_count_subset[disappointed] : -2.3551
wo

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [123]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.8693004559635229

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [126]:
simple_model.coefficients.print_rows(num_rows=21)

+-------------------+--------------+-------+-----------------+-----------------+
|        name       |    index     | class |      value      |      stderr     |
+-------------------+--------------+-------+-----------------+-----------------+
|    (intercept)    |     None     |   1   |   1.2995449552  | 0.0120888541331 |
| word_count_subset | disappointed |   1   |  -2.35509250061 | 0.0504149888557 |
| word_count_subset |     love     |   1   |  1.36543549368  | 0.0303546295109 |
| word_count_subset |    little    |   1   |  0.520628636025 | 0.0214691475665 |
| word_count_subset |    loves     |   1   |  1.67727145556  | 0.0482328275384 |
| word_count_subset |   product    |   1   | -0.320555492996 | 0.0154311321362 |
| word_count_subset |     well     |   1   |  0.504256746398 |  0.021381300631 |
| word_count_subset |    great     |   1   |  0.94469126948  | 0.0209509926591 |
| word_count_subset |     easy     |   1   |  1.19366189833  |  0.029288869202 |
| word_count_subset |     wo

Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [125]:
simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)

+-------------------+--------------+-------+-----------------+-----------------+
|        name       |    index     | class |      value      |      stderr     |
+-------------------+--------------+-------+-----------------+-----------------+
| word_count_subset |    loves     |   1   |  1.67727145556  | 0.0482328275384 |
| word_count_subset |   perfect    |   1   |  1.51448626703  |  0.049861952294 |
| word_count_subset |     love     |   1   |  1.36543549368  | 0.0303546295109 |
|    (intercept)    |     None     |   1   |   1.2995449552  | 0.0120888541331 |
| word_count_subset |     easy     |   1   |  1.19366189833  |  0.029288869202 |
| word_count_subset |    great     |   1   |  0.94469126948  | 0.0209509926591 |
| word_count_subset |    little    |   1   |  0.520628636025 | 0.0214691475665 |
| word_count_subset |     well     |   1   |  0.504256746398 |  0.021381300631 |
| word_count_subset |     able     |   1   |  0.191438302295 | 0.0337581955697 |
| word_count_subset |     ol

Consider the coefficients of **simple_model**. There are 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [129]:
simple_model.coefficients['index']

dtype: str
Rows: 21
[None, 'disappointed', 'love', 'little', 'loves', 'product', 'well', 'great', 'easy', 'work', 'would', 'able', 'perfect', 'old', 'car', 'money', 'waste', 'even', 'less', 'broke', 'return']

In [144]:
#computing how many of the 20 coefficients in the simple_model are nonnegative in the sentiment_model
counter =0
for term in simple_model.coefficients['index']:
    if term != None:
        if sentiment_model.coefficients[sentiment_model.coefficients['index'] ==  term]['value'][0] >=0:
            counter +=1
print(counter)
        


11


In [ ]:
sentiment_model

In [188]:
counter2=0
positive_significant_words = simple_model.coefficients[simple_model.coefficients['value'] >= 0]['index']

print('number of positive significant words in simple_model =',len(positive_significant_words))
for word in positive_significant_words:
    if word != None:
        if float(sentiment_model.coefficients[sentiment_model.coefficients['index'] == word]['value'][0]) >=0:
            counter2 +=1
            print(float(sentiment_model.coefficients[sentiment_model.coefficients['index'] == word]['value'][0]))
print('of those words only', counter2, 'are positive in sentiment_model')


('number of positive significant words in simple_model =', 11)
0.840505732062
0.409931627258
0.974982312515
0.401075574924
0.778953288381
0.734982625568
0.107528021915
1.04479942041
0.0796749090099
0.119657876508
('of those words only', 10, 'are positive in sentiment_model')


# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [175]:
train_results = train_data.copy()
train_results['sentiment_model_pred'] = sentiment_model.predict(train_data)
train_results['simple_model_pred'] = simple_model.predict(train_data)

In [178]:
simple_model_train_acc = len(train_results[train_results['sentiment'] == train_results['simple_model_pred']]) / len(train_results)

sentiment_model_train_acc = len(train_results[train_results['sentiment'] == train_results['sentiment_model_pred']]) / len(train_results)

print('sentiment_model accuracy on train data = ', sentiment_model_train_acc)
print('simple_model accuracy on train data =', simple_model_train_acc)

('sentiment_model accuracy on train data = ', 0.976494573364514)
('simple_model accuracy on train data =', 0.8668150746537147)


Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [181]:
test_results = test_data.copy()
test_results['sentiment_model_pred'] = sentiment_model.predict(test_data)
test_results['simple_model_pred'] = simple_model.predict(test_data)

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [182]:
simple_model_test_acc = len(test_results[test_results['sentiment'] == test_results['simple_model_pred']]) / len(test_results)

sentiment_model_test_acc = len(test_results[test_results['sentiment'] == test_results['sentiment_model_pred']]) / len(test_results)

print('sentiment_model accuracy on test data = ', sentiment_model_test_acc)
print('simple_model accuracy on test data =', simple_model_test_acc)

('sentiment_model accuracy on test data = ', 0.9221862251019919)
('simple_model accuracy on test data =', 0.8693004559635229)


## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [183]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print(num_positive)
print(num_negative)

112164
21252


In [184]:
num_positive/len(train_data)

0.8407087605684476

Now compute the accuracy of the majority class classifier on **test_data**.

In [186]:
num_positive_test = (test_data['sentiment'] == +1).sum()
num_positive_test/len(test_data)

0.8427825773938085